# # Part2. 웹 크롤링으로 업소별 1인분 평균가격 불러오기
- 네이버 지도에 상호명을 검색했을때 나오는 메뉴와 가격을 크롤링해 가게별 평균 소비가를 계산하려고함
- 메뉴 가격 크롤링 (Selenium)과 배치 크롤링 코드 파일 따로 생성 후 커맨드창으로 크롤링 돌리겠음
- 네이버 지도 iframe 구조 분하기석
- 메뉴 사진/텍스트 존재 여부에 따른 분기 , 예외처리 포함처리메뉴 수 0인 가게 → 업종별 평균가로 대체 예정
- 엽떡, 치킨같이 여러명이서 먹어야만 하는 업소 메뉴는 총메뉴가격/메뉴수로 했을 때 1인당 소비가격이 엽떡 한그릇 가격, 치킨한마리 가격으로 잡힘
- 이러한 메뉴들 업소 분류에서 분류해 놓아야 했었음 나누는 방식 차별점 혹은 평균가격 다른 측정 필요 (팀원들과 논의 필요)체`) 포함

# # 셀레니움(selenium)
- 크롤링이나 테스트 자동화에 쓰이는 패키지(라이브러리)
- 브라우저 열기, 페이지 이동, 요소 찾기, 버튼 클릭, 키 입력 등에 사용

## 1) inha_crawling_util.py
- 네이버 지도에서 가게 상호명을 검색하여 메뉴 이름과 가격을 크롤링하는 함수

### 1. 라이브러리 불러오기

In [ ]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# time : 코드 실행을 잠깐 멈춤 (sleep)
# Keys : 키보드 입력 흉내냄 (ENTER, TAB 등)
# WebDriverWait : 요소가 나타날 때까지 기다리는 기능
# EC (expected_conditions) : 어떤 조건을 만족하면 다음으로 넘어감
# TimeoutException : 시간 초과 에러 처리용

- store_name: 검색할 가게 이름
- driver: Selenium의 Chrome WebDriver 객체
- 결과를 저장할 딕셔너리 초기화: 메뉴 정보는 리스트로 (메뉴명, 가격) 형식의 튜플로 저장

In [ ]:
def crawling_get_menus(store_name, driver):
   menu_data = {
        "place_name": store_name,
        "menu": []
    }

### 2. 네이버 지도에서 가게명 검색
- 네이버 지도 검색 페이지에 접속해서 가게 이름으로 검색
- 페이지가 SPA(싱글 페이지 애플리케이션)이라 로딩을 기다리기 위해 time.sleep 사용
- 만약, 네이버 지도의 구조가 바뀌면 CSS 선택자나 iframe 구조도 같이 수정해야 작동함

In [ ]:
    driver.get("https://map.naver.com/v5/search/" + store_name)
    time.sleep(2.5)  # 페이지 로딩 대기

### 2. 검색 결과 iframe으로 진입
- 네이버 지도는 검색 결과를 iframe에 표시함
- searchIframe이 로딩될 때까지 기다렸다가 진입

In [ ]:
    try:
        WebDriverWait(driver, 2).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe"))
        )
    except TimeoutException:
        print("searchIframe 로딩 실패")
        return menu_data

### 3. 검색 결과 리스트 로딩 대기
-  검색 결과들이 나오는 리스트(ul 태그)가 완전히 로딩될 때까지 대기

In [ ]:
    try:
        container = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul'))
        )
    except TimeoutException:
        print("검색 리스트 컨테이너 로딩 실패")
        return menu_data

### 4. 검색 결과 첫 번째 클릭
- 검색 결과 중 첫 번째 가게 클릭 (상세 페이지 이동)
- 일부 결과는 이미지(썸네일)용 a 태그일 수도 있어서, "place_thumb" 클래스 체크 후 적절한 버튼 선택

In [ ]:
    buttons = container.find_elements(By.TAG_NAME, 'a')
    if not buttons:
        print("검색 결과 없음")
        return menu_data
# 썸네일(a)일 경우 두 번째 버튼 클릭
    first_btn_class = buttons[0].get_attribute("class")
    if "place_thumb" in first_btn_class and len(buttons) > 1:
        buttons[1].send_keys(Keys.ENTER) # 엔터키 입력으로 첫 번째 검색 결과를 클릭
    else:
        buttons[0].send_keys(Keys.ENTER)

### 5. 상세 페이지 iframe으로 전환
- 검색 결과를 클릭하면 페이지가 바뀌고, 또 다른 iframe(entryIframe)으로 전환됨
- 이 iframe 안에서 메뉴 정보를 찾을 수 있음............................

In [ ]:
    driver.switch_to.default_content()
    try:
        entry_iframe = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, "entryIframe"))
        )
        driver.switch_to.frame(entry_iframe)
    except TimeoutException:
        print("entryIframe 로딩 실패")
        return menu_data

### 6. 메뉴 섹션 로딩 유도 (스크롤)
- 메뉴 섹션이 스크롤을 내려야 보이기 때문에, 자바스크립트를 이용해 강제로 내림
- place_section Sv9Ys 클래스를 가진 메뉴 영역 로딩 대기

In [ ]:
    driver.execute_script("window.scrollTo(0, 700)")
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.place_section.Sv9Ys"))
        )
    except TimeoutException:
        print("메뉴 섹션 로딩 실패")
        return menu_data

### 7. 사진 포함 메뉴 (ul.t1osG > li.ipNNM)
- 사진이 있는 메뉴는 ul.t1osG > li.ipNNM 구조로 되어 있음
- 각 메뉴 항목에서 메뉴 이름 (span.VQvNX)과 가격 (div.gl2cc)을 추출

In [ ]:
    photo_menu_list = driver.find_elements(By.CSS_SELECTOR, "ul.t1osG")
    if photo_menu_list:
        li_elements = photo_menu_list[0].find_elements(By.CSS_SELECTOR, "li.ipNNM")
        for li in li_elements:
            menu_name = li.find_element(By.CSS_SELECTOR, "span.VQvNX").text
            menu_price = li.find_element(By.CSS_SELECTOR, "div.gl2cc").text
            menu_data["menu"].append((menu_name, menu_price))
        return menu_data

### 8. 사진 없는 텍스트 메뉴 (ul.jnwQZ > li.gHmZ_)
- 사진 없이 텍스트로만 구성된 메뉴는 ul.jnwQZ > li.gHmZ_ 구조
- 메뉴명: div.ds3HZ a, 가격: div.mkBm3
- 사진 메뉴도 텍스트 메뉴도 없는 경우는 메뉴 정보가 없는 것으로 간주

In [ ]:
    text_menu_list = driver.find_elements(By.CSS_SELECTOR, "ul.jnwQZ")
    if text_menu_list:
        li_elements = text_menu_list[0].find_elements(By.CSS_SELECTOR, "li.gHmZ_")
        for li in li_elements:
            menu_name = li.find_element(By.CSS_SELECTOR, "div.ds3HZ a").text
            menu_price = li.find_element(By.CSS_SELECTOR, "div.mkBm3").text
            menu_data["menu"].append((menu_name, menu_price))
        return menu_data

    # 9. 메뉴 정보가 없을 경우
    print("메뉴 정보 없음")
    return menu_data

## 2) inha_batch_crawler.py
- 배치 처리 루프 코드

### 1. 라이브러리 불러오기

In [ ]:
import pandas as pd
import time # 여기서는 사용 안 되긴 함
import os # 디렉토리 생성, 파일 경로 처리용
from selenium import webdriver # 브라우저 조작 도구
from selenium.webdriver.chrome.options import Options # 크롬 설정
from inha_crawling_util import crawling_get_menus  # 만든 크롤링 함수 import

### 2. headless 모드로 크롬 드라이버 설정
- headless 모드: 브라우저를 눈에 안 띄게 백그라운드에서 실행
- webdriver.Chrome(options=...) : 이 설정을 적용한 크롬 드라이버 실행

In [ ]:
options = Options()
options.add_argument("--headless") # 창 안 띄우는 모드
options.add_argument("--disable-gpu")  # GPU 끔
driver = webdriver.Chrome(options=options)

### 3. 상호명 리스트 불러오기

In [ ]:
df = pd.read_csv("C:/Users/sj123/KSEB_Project/KSEB_Project_output/inha_restaurant_FE.csv", encoding="utf-8-sig")
store_names = df["상호명"].tolist()
results = []

### 4. 크롤링 루프

In [ ]:
for name in store_names:
    print(f"크롤링 중: {name}") 

### 5. 메뉴 데이터 가져와서 평균 계산하기
- 메뉴 리스트에서 가격을 꺼내 숫자(int)로 바꾸기 : "4,500원" → 4500

In [ ]:
    try:
        data = crawling_get_menus(name, driver) # 메뉴 정보 크롤링
        prices = []
        for menu_name, price in data["menu"]:
            try:
                num = int(price.replace(",", "").replace("원", "").strip())
                prices.append(num)
            except:
                continue

        
        avg_price = sum(prices) // len(prices) if prices else None # 평균 계산

### 6. 결과 저장 리스트에 추가
- 각 가게마다 {상호명, 평균가격, 메뉴 수} 형태로 저장

In [ ]:
        results.append({
            "상호명": name,
            "1인 평균 가격": avg_price,
            "메뉴 수": len(prices)
        })

### 7. 에러 발생 시 처리
- 크롤링 실패해도 멈추지 않도록 try-except로 감싸서 예외 처리

In [ ]:
    except Exception as e:
        print(f"{name} 처리 중 오류 발생: {e}")
        results.append({
            "상호명": name,
            "1인 평균 가격": None,
            "메뉴 수": 0
        })

### 8. 결과 CSV로 저장
- 결과 리스트 - DataFrame으로 변환
- 일단 CSV 파일로 저장 후 확인 예정

In [ ]:
os.makedirs("KSEB_Project_output", exist_ok=True)
pd.DataFrame(results).to_csv("KSEB_Project_output/inha_result_final.csv", index=False, encoding="utf-8-sig")
driver.quit() # 크롬 드라이버 종료
print("크롤링 완료 및 저장")

## 3) 커맨드창 파이썬 폴더에서 코드 실행

In [ ]:
python inha_batch_crawler.py